## Ewaluacja trajektorii
Ewaluacja trajektorii porównuje ciąg kroków rozumowania modelu (prediction) z wzorcową sekwencją (reference), oceniając zgodność merytoryczną, kolejność i kompletność poszczególnych etapów. Pozwala wykryć nie tylko błędną odpowiedź końcową, ale też gdzie w łańcuchu rozumowania pojawiło się odchylenie (np. pominięty lub zniekształcony step), dzięki czemu jest szczególnie użyteczna przy agentach i zadaniach wieloetapowych.

### Instalacja bibliotek

In [1]:
!pip install langchain-core langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### Ewaluacja trajektorii

In [1]:
from langchain_classic.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import json

load_dotenv()

#  LLM używany przez ewaluator
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

evaluator = load_evaluator("trajectory", llm=llm)

reference_steps = [
    "Fibonacci zaczyna się od 0, 1",
    "Kolejne liczby to suma dwóch poprzednich",
    "5. liczba to 5"
]

prediction_steps = [
    "Fibonacci zaczyna się od 0, 1",
    "Kolejne liczby to suma poprzednich",
    "5. liczba to 4"
]

result = evaluator.invoke({
    "question": "Jaka jest 5. liczba ciągu Fibonacciego?",
    "answer": "4",
    "agent_trajectory": prediction_steps,
    "reference": {
        "answer": "5",
        "agent_trajectory": reference_steps
    }
})

print(json.dumps(result, indent=4))

{
    "question": "Jaka jest 5. liczba ci\u0105gu Fibonacciego?",
    "answer": "4",
    "agent_trajectory": [
        "Fibonacci zaczyna si\u0119 od 0, 1",
        "Kolejne liczby to suma poprzednich",
        "5. liczba to 4"
    ],
    "reference": "\n\nThe following is the expected answer. Use this to measure correctness:\n[GROUND_TRUTH]\n{'answer': '5', 'agent_trajectory': ['Fibonacci zaczyna si\u0119 od 0, 1', 'Kolejne liczby to suma dw\u00f3ch poprzednich', '5. liczba to 5']}\n[END_GROUND_TRUTH]\n",
    "score": 0.0,
    "reasoning": "Let's evaluate the AI language model's answer step by step based on the provided criteria:\n\ni. **Is the final answer helpful?**\n   - The final answer given by the AI model is \"4,\" which is incorrect. The 5th number in the Fibonacci sequence is actually \"5.\" Therefore, the answer is not helpful.\n\nii. **Does the AI language model use a logical sequence of tools to answer the question?**\n   - The AI model provides a logical sequence in its r

In [2]:
correct_prediction_steps = [
    "Fibonacci zaczyna się od 0, 1",
    "Kolejne liczby to suma 2 poprzednich",
    "5. liczba to 5"
]

result = evaluator.invoke({
    "question": "Jaka jest 5. liczba ciągu Fibonacciego?",
    "answer": "5",
    "agent_trajectory": correct_prediction_steps,
    "reference": {
        "answer": "5",
        "agent_trajectory": reference_steps
    }
})

print(json.dumps(result, indent=4))

{
    "question": "Jaka jest 5. liczba ci\u0105gu Fibonacciego?",
    "answer": "5",
    "agent_trajectory": [
        "Fibonacci zaczyna si\u0119 od 0, 1",
        "Kolejne liczby to suma 2 poprzednich",
        "5. liczba to 5"
    ],
    "reference": "\n\nThe following is the expected answer. Use this to measure correctness:\n[GROUND_TRUTH]\n{'answer': '5', 'agent_trajectory': ['Fibonacci zaczyna si\u0119 od 0, 1', 'Kolejne liczby to suma dw\u00f3ch poprzednich', '5. liczba to 5']}\n[END_GROUND_TRUTH]\n",
    "score": 1.0,
    "reasoning": "Let's evaluate the AI language model's answer step by step based on the provided criteria:\n\ni. **Is the final answer helpful?**\n   - Yes, the final answer \"5\" is correct and directly answers the question about the 5th number in the Fibonacci sequence.\n\nii. **Does the AI language model use a logical sequence of tools to answer the question?**\n   - Yes, the sequence of statements in the agent trajectory logically explains the Fibonacci sequ